In [1]:
!nvidia-smi
from google.colab import drive
drive.mount('/content/drive')
!ls drive/'My Drive'/'Colab Notebooks'//bad_chatbot
!pip install -r drive/'My Drive'/'Colab Notebooks'/bad_chatbot/Bert_Gpt_Chatbot/requirements.txt
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
import sys
sys.path.append('drive/My Drive')
sys.path.append('drive/My Drive/Colab Notebooks/')
sys.path.append('drive/My Drive/Colab Notebooks/bad_chatbot')
!pip install sentencepiece
!pip install transformers==4.16.2

Mon Feb 28 13:54:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch, os, gc
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

from Bert_Gpt_Chatbot.dataloader.dataloader import BERTDataset
from Bert_Gpt_Chatbot.model.kobert_model import BERTClassifier

In [3]:
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

root_path='drive/My Drive/Colab Notebooks/bad_chatbot/Bert_Gpt_Chatbot'


dataset_train = []
dataset_test = []

df = pd.read_csv(f'{root_path}/input/datasets/upsampling_final_train_test_datasets_normalize_0228.csv',sep='|')

train_df = pd.DataFrame()
test_df = pd.DataFrame()
categories = ['혐오', '섹슈얼', '일반_긍정']
for i in categories:
    filter_df = df[df['카테고리'] == i]
    train_set, test_set = train_test_split(filter_df, test_size=0.2)  # test_size = 비율설정
    train_df = pd.concat([train_df, train_set], ignore_index=True)
    test_df = pd.concat([test_df, test_set], ignore_index=True)

dataset_train = []
dataset_test = []

for i in range(len(train_df)) :
    if train_df.iloc[i,1] == '일반_긍정' :
        train_df.iloc[i,1] = '0'
    elif train_df.iloc[i,1] == '섹슈얼' :
        train_df.iloc[i,1] = '1'
    else : train_df.iloc[i,1] = '2'

    dataset_train.append([train_df.iloc[i,2], train_df.iloc[i,1]])

for i in range(len(test_df)) :
    if test_df.iloc[i,1] == '일반_긍정' :
        test_df.iloc[i,1] = '0'
    elif test_df.iloc[i,1] == '섹슈얼' :
        test_df.iloc[i,1] = '1'
    else : test_df.iloc[i,1] = '2'

    dataset_test.append([test_df.iloc[i,2], test_df.iloc[i,1]])


print(len(dataset_train))
print(len(dataset_test))

5394
1350


In [4]:
max_len = 64
batch_size = 8
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
learning_rate = 5e-5
add_token = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
             'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ', 'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']


train_dataset = BERTDataset(dataset_train, add_token=add_token)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)

test_dataset = BERTDataset(dataset_test, add_token=add_token)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=True)

model = BERTClassifier(vocab_size=train_dataset.vocab_size, add_token=train_dataset.added_token_num)
model.to(device)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [7]:
checkpoint_path =f"{root_path}/output"
save_ckpt_path = f'{checkpoint_path}/model/cp_KoBert.pt'

pre_epoch, pre_loss, train_step = 0, 0, 0
if os.path.isfile(save_ckpt_path):
    checkpoint = torch.load(save_ckpt_path, map_location=device)
    
    pre_epoch = checkpoint['Epoch']
    model.load_state_dict(checkpoint['State_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    # pre_loss = checkpoint['Loss']
    # train_step =  checkpoint['Train_no']
    # total_train_step =  checkpoint['Total_train_step']

    print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")  #, loss={pre_loss}\n")
    # best_epoch += 1

In [8]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

best_model = 0

In [9]:
for epoch in range(num_epochs):

    train_acc = 0.0
    test_acc = 0.0

    model.train()
    for batch_id, (input_ids, token_type_ids, attention_mask, label) in enumerate(tqdm(train_dataloader)):
        input_ids = input_ids.long().to(device)
        token_type_ids = token_type_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.long().to(device)

        optimizer.zero_grad()
        out = model(input_ids, token_type_ids, attention_mask)

        del input_ids
        del token_type_ids
        del attention_mask


        loss = loss_fn(out, label)
        del label
        del out
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()

    print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

    model.eval()
    with torch.no_grad() :
        for batch_id, (input_ids, token_type_ids, attention_mask, label) in enumerate(tqdm(test_dataloader)):
            input_ids = input_ids.long().to(device)
            token_type_ids = token_type_ids.long().to(device)
            attention_mask = attention_mask.long().to(device)
            label = label.long().to(device)

            optimizer.zero_grad()
            out = model(input_ids, token_type_ids, attention_mask)

            test_acc += calc_accuracy(out, label)

    print("epoch {} test acc {}".format(epoch + 1, test_acc / (batch_id + 1)))


    if best_model < test_acc:
        
        state = {'Epoch': epoch,
                 'State_dict': model.state_dict(),
                 'optimizer': optimizer.state_dict()}
        torch.save(state, f'{checkpoint_path}/model/cp_KoBert.pt')         
        torch.save(model, f'{checkpoint_path}/model/KoBert_3multi_{test_acc/(batch_id + 1)}.pt')

        best_model = test_acc

100%|██████████| 675/675 [00:51<00:00, 13.06it/s]


epoch 1 train acc 0.0


100%|██████████| 169/169 [00:03<00:00, 48.91it/s]


epoch 1 test acc 0.8217455621301775


100%|██████████| 675/675 [00:51<00:00, 13.05it/s]


...

epoch 9 test acc 0.9068047337278107


100%|██████████| 675/675 [00:51<00:00, 13.12it/s]


epoch 10 train acc 0.0


100%|██████████| 169/169 [00:03<00:00, 49.08it/s]

epoch 10 test acc 0.908284023668639


In [ ]:
### 코드 참조

# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from IPython.display import display
# from tqdm import tqdm

# import torch
# from transformers import AdamW
# from torch.utils.data import dataloader
# from final_project.dataloader.wellness import WellnessTextClassificationDataset
# from final_project.model.kobert import KoBERTforSequenceClassfication

In [ ]:
# root_path='drive/My Drive/Colab Notebooks/final_project'
# data_path = f"{root_path}/data/test_KOBERT.txt"
# checkpoint_path =f"{root_path}/checkpoint"
# save_ckpt_path = f"{checkpoint_path}/test_KOBERT.pth"

# n_epoch = 10          # Num of Epoch
# batch_size = 8      # 배치 사이즈
# ctx = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device(ctx)
# save_step = 100 # 학습 저장 주기
# learning_rate = 5e-6  # Learning Rate

# # WellnessTextClassificationDataset 데이터 로더
# dataset = WellnessTextClassificationDataset(file_path=data_path, device=device)
# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# model = KoBERTforSequenceClassfication()
# model.to(device)

# # Prepare optimizer and schedule (linear warmup and decay)
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#       'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]
# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# pre_epoch, pre_loss, train_step = 0, 0, 0
# if os.path.isfile(save_ckpt_path):
#     checkpoint = torch.load(save_ckpt_path, map_location=device)
#     pre_epoch = checkpoint['epoch']
#     # pre_loss = checkpoint['loss']
#     train_step =  checkpoint['train_step']
#     total_train_step =  checkpoint['total_train_step']

#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

#     print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")  #, loss={pre_loss}\n")
#     # best_epoch += 1

# losses = []
# offset = pre_epoch
# for step in range(n_epoch):
#     epoch = step + offset
#     loss = train(device, epoch, model, optimizer, train_loader, save_step, save_ckpt_path, train_step)
#     losses.append(loss)

# # data
# data = {
#     "loss": losses
# }
# df = pd.DataFrame(data)
# display(df)

# # graph
# plt.figure(figsize=[12, 4])
# plt.plot(losses, label="loss")
# plt.legend()
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.show()